In [2]:
import requests
from bs4 import BeautifulSoup as bs
import json
import numpy as np
import pandas as pd
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [3]:
# 1. 카테고리 선택하기
category = ['epl', 'primera', 'bundesliga', 'seria', 'ligue1',
            'denmark', 'champs', 'europa', 'uecl', 'facup',
            'carlingcup', 'copadelrey', 'clubworldcup']

cate = input("카테고리를 선택하시오(영어): ")
schedule_url = f'https://sports.news.naver.com/wfootball/schedule/index?category={cate}'
response_1 = requests.get(schedule_url, headers=headers)
soup_1 = bs(response_1.text, 'html.parser')
soup_1_1 = soup_1.select('script')[-5]

match_schedule = re.search(r"(?<=scoreboardModel: ){.*?}(?=,\n)", soup_1_1.text).group()
match_info = json.loads(match_schedule)['scoreboardList']

hometeamName = []
awayteamName = []
gameId = []
for i in match_info:
    hometeamName.append(i['homeTeamName'])
    awayteamName.append(i['awayTeamName'])
    gameId.append(i['gameId'])
    
gameInfo = {
    'HOME':hometeamName,
    'AWAY':awayteamName,
    'ID':gameId
}
df_gameInfo = pd.DataFrame(gameInfo)

team = input("팀 이름을 입력하세요(한글): ")
if team in hometeamName:
    for i in range(6):
        if df_gameInfo.HOME[i] != team:
            pass
        else:
            gameInfoID = df_gameInfo.ID[i]
elif team in awayteamName:
    for i in range(6):
        if df_gameInfo.AWAY[i] != team:
            pass
        else:
            gameInfoID = df_gameInfo.ID[i]
else:
    print("예정된 경기가 없습니다.")


# 2. (경기 시작 전) 선발 라인업 정보 가져오기
# lineup_url = 'https://m.sports.naver.com/game/2023081465041892243/lineup'
lineup_url = 'https://api-gw.sports.naver.com/schedule/games/2023050110034152359/lineup'
response_2 = requests.get(lineup_url, headers=headers)
soup_2 = json.loads(response_2.text)

# 홈팀 라인업데이터
h_lineup_data = soup_2['result']['lineUpData']['lineup']['home']['players']['lineup']

# 홈팀 포메이션
h_formation = soup_2['result']['lineUpData']['lineup']['home']['formation'].split('-')

# 홈팀 라인업
home_start = []
for i in range(1):
    home_start.append(h_lineup_data[0][i]['name'])
for i in range(4):
    home_start.append(h_lineup_data[1][i]['name'])
for i in range(3):
    home_start.append(h_lineup_data[2][i]['name'])
for i in range(3):
    home_start.append(h_lineup_data[3][i]['name'])

# 어웨이팀 라인업데이터
a_lineup_data = soup_2['result']['lineUpData']['lineup']['away']['players']['lineup']

# 어웨이팀 포메이션
a_formation = soup_2['result']['lineUpData']['lineup']['away']['formation'].split('-')

# 어웨이팀 라인업
away_start = []
for i in range(1):
    away_start.append(a_lineup_data[0][i]['name'])
for i in range(2):
    away_start.append(a_lineup_data[1][i]['name'])
for i in range(4):
    away_start.append(a_lineup_data[2][i]['name'])
for i in range(3):
    away_start.append(a_lineup_data[3][i]['name'])
for i in range(1):
    away_start.append(a_lineup_data[4][i]['name'])

# 홈, 어웨이 선발명단
starting_player = {
    'Home':home_start,
    'Away':away_start[::-1]
}
df_starting_player = pd.DataFrame(starting_player)



# 4. (경기 종료 후) 최종 스코어와 세부 기록 가져오기
record_url = 'https://api-gw.sports.naver.com/schedule/games/2023050110034152359'
response_4 = requests.get(record_url, headers=headers)
soup_4 = json.loads(response_4.text)

# 최종 스코어
result_data = soup_4['result']['game']
# print(result_data['homeTeamName'], result_data['homeTeamScore'], result_data['awayTeamName'], result_data['awayTeamScore'])

# 홈팀이 득점한 시간
home_t = list(t['time'] for t in result_data['scorers']['home'])
home_at = list(a_t['addedTime'] for a_t in result_data['scorers']['home'])

# 홈팀 득점한 선수
home_player = list(p['playerName'] for p in result_data['scorers']['home'])

# 어웨이팀이 득점한 시간
away_t = list(t['time'] for t in result_data['scorers']['away'])
away_at = list(a_t['addedTime'] for a_t in result_data['scorers']['away'])

# 어웨이팀 득점한 선수
away_player = list(p['playerName'] for p in result_data['scorers']['away'])

# 홈팀 경기 정보
home_data = {
    'Name':home_player,
    't':home_t,
    'at':home_at
}
df_h = pd.DataFrame(home_data)
HomeFinalInfo = df_h.fillna(0)
HomeFinalInfo["Time"] = HomeFinalInfo['at'] + HomeFinalInfo['t']
# print(HomeFinalInfo)

# 어웨이팀 경기 정보
away_data = {
    'Name':away_player,
    't':away_t,
    'at':away_at
}
df_a = pd.DataFrame(away_data)
AwayFinalInfo = df_a.fillna(0)
AwayFinalInfo["Time"] = AwayFinalInfo['at'] + AwayFinalInfo['t']
# print(AwayFinalInfo)

In [14]:
# 1. 카테고리 선택하기
category = ['epl', 'primera', 'bundesliga', 'seria', 'ligue1',
            'denmark', 'champs', 'europa', 'uecl', 'facup',
            'carlingcup', 'copadelrey', 'clubworldcup']

cate = input("카테고리를 선택하시오(영어): ")
schedule_url = f'https://sports.news.naver.com/wfootball/schedule/index?category={cate}'
response_1 = requests.get(schedule_url, headers=headers)
soup_1 = bs(response_1.text, 'html.parser')
soup_1_1 = soup_1.select('script')[-5]

match_schedule = re.search(r"(?<=scoreboardModel: ){.*?}(?=,\n)", soup_1_1.text).group()
match_info = json.loads(match_schedule)['scoreboardList']

hometeamName = []
awayteamName = []
gameId = []
for i in match_info:
    hometeamName.append(i['homeTeamName'])
    awayteamName.append(i['awayTeamName'])
    gameId.append(i['gameId'])
    
gameInfo = {
    'HOME':hometeamName,
    'AWAY':awayteamName,
    'ID':gameId
}
df_gameInfo = pd.DataFrame(gameInfo)

# 정보를 알고싶은 팀 정하기
team = input("팀 이름을 입력하세요(한글): ")
if team in hometeamName:
    for i in range(6):
        if df_gameInfo.HOME[i] != team:
            pass
        else:
            gameInfoID = df_gameInfo.ID[i]
elif team in awayteamName:
    for i in range(6):
        if df_gameInfo.AWAY[i] != team:
            pass
        else:
            gameInfoID = df_gameInfo.ID[i]
else:
    print("예정된 경기가 없습니다.")

# 2. (경기 시작 전) 선발 라인업 정보 가져오기
lineup_url = f'https://api-gw.sports.naver.com/schedule/games/{gameInfoID}/lineup'
response_2 = requests.get(lineup_url, headers=headers)
soup_2 = json.loads(response_2.text)

if bool(soup_2['result']['lineUpData']) == False:
    print("라인업 정보가 업로드되지 않았습니다.")
else:
    # 홈팀 라인업데이터
    h_lineup_data = soup_2['result']['lineUpData']['lineup']['home']['players']['lineup']

    # 홈팀 포메이션
    h_formation = soup_2['result']['lineUpData']['lineup']['home']['formation'].split('-') 
    int_h_formation = list(map(int, h_formation))
    int_h_formation.insert(0, 1)

    # 홈팀 라인업
    home_start = []
    

라인업 정보가 업로드되지 않았습니다.


In [4]:
soup_1_1

<script type="text/javascript">
	new naver.sports.schedule.SchedulePage({
		template: {
			seasonListTemplate : "_wfootballSeasonListTemplate"
		},
		today: '20230806',
		defaultDateId: '20230812',
		section: 'wfootball',
		category: 'epl',
		seasonList: [{"startYear":"2023","title":"2023/24","seasonName":"2023-24","yearMonthList":["202308","202309","202310","202311","202312","202401","202402","202403","202404","202405"],"yearMonthModelList":[{"year":"2023","month":"01","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"02","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"03","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"04","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"05","

In [5]:
monthly = re.search(r"(?<=scheduleData: ){.*?}(?=\n)", soup_1_1.text).group()
days = json.loads(monthly)["monthlyScheduleDailyGroup"]
days

[{'date': '20230801', 'scheduleList': [], 'empty': True},
 {'date': '20230802', 'scheduleList': [], 'empty': True},
 {'date': '20230803', 'scheduleList': [], 'empty': True},
 {'date': '20230804', 'scheduleList': [], 'empty': True},
 {'date': '20230805', 'scheduleList': [], 'empty': True},
 {'date': '20230806', 'scheduleList': [], 'empty': True},
 {'date': '20230807', 'scheduleList': [], 'empty': True},
 {'date': '20230808', 'scheduleList': [], 'empty': True},
 {'date': '20230809', 'scheduleList': [], 'empty': True},
 {'date': '20230810', 'scheduleList': [], 'empty': True},
 {'date': '20230811', 'scheduleList': [], 'empty': True},
 {'date': '20230812',
  'scheduleList': [{'homeTeamName': '번리',
    'awayTeamName': '맨시티',
    'homeTeamScore': '0',
    'awayTeamScore': '0',
    'homeTeamEmblem64URI': 'https://dthumb-phinf.pstatic.net?src=https://sports-phinf.pstatic.net/team/wfootball/default/70.png&type=f64_64',
    'awayTeamEmblem64URI': 'https://dthumb-phinf.pstatic.net?src=https://spor

In [39]:
matchDay = []
hometeamName = []
awayteamName = []

for i in days:
    # 경기가 없는 날
    if i['empty'] == True:
        pass
    # 경기가 있는 날
    else:
        print(i['date'])
        for j in i['scheduleList']:
            print(j['homeTeamName'])
        # matchDay.append(i['date'])
        # for j in i['scheduleList']:
        #     print(j)

20230812
번리
아스널
본머스
브라이튼
에버턴
셰필드
20230813
뉴캐슬
브렌트포드
20230814
첼시
20230815
맨유
20230819
노팅엄
풀럼
리버풀
루턴
울버햄튼
20230820
토트넘
맨시티
아스톤 빌라
20230821
웨스트햄
20230822
팰리스
20230826
첼시
본머스
아스널
브렌트포드
에버턴
맨유
20230827
브라이튼
번리
셰필드
20230828
뉴캐슬


In [23]:
import datetime

now = datetime.datetime.today()
today = now.strftime('%Y%m%d')
today

'20230806'